# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0     # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),        # Fill input shape
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Fill number of hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Fill number of output neurons
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Fill name of loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end - start} seconds")       # Output training time
model.evaluate(x_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.8582 - loss: 0.5008
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9554 - loss: 0.1521
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9686 - loss: 0.1055
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9752 - loss: 0.0831
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9796 - loss: 0.0683
TF Training time: 58.095083475112915 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9681 - loss: 0.1060


[0.09224318712949753, 0.9718000292778015]

## Convert TensorFlow model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

## PyTorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(___, ___)    # Fill correct input and output size
        self.fc2 = nn.Linear(___, ___)    # Fill correct input and output size
    def forward(self, x):
        x = F.relu(self.___(x))    # Fill correct layer
        return self.___(x)    # Fill correct layer

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

## Convert PyTorch model to ONNX

In [ ]:
# Install ONNX
!pip install onnx

In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3622, Accuracy: 0.0625
Step 100, Loss: 0.6295, Accuracy: 0.7296
Step 200, Loss: 0.5923, Accuracy: 0.8033
Step 300, Loss: 0.2255, Accuracy: 0.8341
Step 400, Loss: 0.3326, Accuracy: 0.8506
Step 500, Loss: 0.3347, Accuracy: 0.8618
Step 600, Loss: 0.2799, Accuracy: 0.8718
Step 700, Loss: 0.4746, Accuracy: 0.8785
Step 800, Loss: 0.1422, Accuracy: 0.8848
Step 900, Loss: 0.4233, Accuracy: 0.8890
Step 1000, Loss: 0.0638, Accuracy: 0.8931
Step 1100, Loss: 0.1412, Accuracy: 0.8964
Step 1200, Loss: 0.1497, Accuracy: 0.8992
Step 1300, Loss: 0.0914, Accuracy: 0.9011
Step 1400, Loss: 0.2254, Accuracy: 0.9038
Step 1500, Loss: 0.1605, Accuracy: 0.9064
Step 1600, Loss: 0.2107, Accuracy: 0.9085
Step 1700, Loss: 0.1934, Accuracy: 0.9108
Step 1800, Loss: 0.0694, Accuracy: 0.9129
Training Accuracy for epoch 1: 0.9146

Epoch 2/5
Step 0, Loss: 0.1071, Accuracy: 0.9688
Step 100, Loss: 0.2680, Accuracy: 0.9477
Step 200, Loss: 0.2089, Accuracy: 0.9468
Step 300, Loss: 0.1341, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.2920, Accuracy: 0.2188
Step 100, Loss: 0.4769, Accuracy: 0.7358
Step 200, Loss: 0.2769, Accuracy: 0.8089
Step 300, Loss: 0.2126, Accuracy: 0.8365
Step 400, Loss: 0.3269, Accuracy: 0.8522
Step 500, Loss: 0.3170, Accuracy: 0.8662
Step 600, Loss: 0.2814, Accuracy: 0.8748
Step 700, Loss: 0.2204, Accuracy: 0.8822
Step 800, Loss: 0.4692, Accuracy: 0.8876
Step 900, Loss: 0.1966, Accuracy: 0.8923
Step 1000, Loss: 0.4621, Accuracy: 0.8958
Step 1100, Loss: 0.3286, Accuracy: 0.8994
Step 1200, Loss: 0.1512, Accuracy: 0.9023
Step 1300, Loss: 0.4084, Accuracy: 0.9042
Step 1400, Loss: 0.2332, Accuracy: 0.9069
Step 1500, Loss: 0.4809, Accuracy: 0.9087
Step 1600, Loss: 0.1113, Accuracy: 0.9115
Step 1700, Loss: 0.2357, Accuracy: 0.9136
Step 1800, Loss: 0.3394, Accuracy: 0.9155
Training Accuracy for epoch 1: 0.9172

Epoch 2/5
Step 0, Loss: 0.0390, Accuracy: 1.0000
Step 100, Loss: 0.4901, Accuracy: 0.9493
Step 200, Loss: 0.2235, Accuracy: 0.9512
Step 300, Loss: 0.1108, Accuracy: